# Day 10: Syntax Scoring

[https://adventofcode.com/2021/day/10](https://adventofcode.com/2021/day/10)

## Description

### Part One

You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

    Syntax error in navigation subsystem on line: all of them

_All of them?!_ The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing _chunks_. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must _open_ and _close_ with one of four legal pairs of matching characters:

*   If a chunk opens with `(`, it must close with `)`.
*   If a chunk opens with `[`, it must close with `]`.
*   If a chunk opens with `{`, it must close with `}`.
*   If a chunk opens with `<`, it must close with `>`.

So, `()` is a legal chunk that contains no other chunks, as is `[]`. More complex but valid chunks include `([])`, `{()()()}`, `<([{}])>`, `[<>({}){}[([])<>]]`, and even `(((((((((())))))))))`.

Some lines are _incomplete_, but others are _corrupted_. Find and discard the corrupted lines first.

A corrupted line is one where a chunk _closes with the wrong character_ - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include `(]`, `{()()()>`, `(((()))}`, and `<([]){()}[{}])`. Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

For example, consider the following navigation subsystem:

    [({(<(())[]>[[{[]{<()<>>
    [(()[<>])]({[<{<<[]>>(
    {([(<{}[<>[]}>{[]{[(<()>
    (((({<>}<{<{<>}{[]{[]{}
    [[<[([]))<([[{}[[()]]]
    [{[{({}]{}}([{[{{{}}([]
    {<[[]]>}<{[{[{[]{()[[[]
    [<(<(<(<{}))><([]([]()
    <{([([[(<>()){}]>(<<{{
    <{([{{}}[<[[[<>{}]]]>[]]


Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

*   `{([(<{}[<>[]}>{[]{[(<()>` - Expected `]`, but found `}` instead.
*   `[[<[([]))<([[{}[[()]]]` - Expected `]`, but found `)` instead.
*   `[{[{({}]{}}([{[{{{}}([]` - Expected `)`, but found `]` instead.
*   `[<(<(<(<{}))><([]([]()` - Expected `>`, but found `)` instead.
*   `<{([([[(<>()){}]>(<<{{` - Expected `]`, but found `>` instead.

Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the _first illegal character_ on the line and look it up in the following table:

*   `)`: `3` points.
*   `]`: `57` points.
*   `}`: `1197` points.
*   `>`: `25137` points.

In the above example, an illegal `)` was found twice (`2*3 = 6` points), an illegal `]` was found once (_`57`_ points), an illegal `}` was found once (_`1197`_ points), and an illegal `>` was found once (_`25137`_ points). So, the total syntax error score for this file is `6+57+1197+25137 = 26397` points!

Find the first illegal character in each corrupted line of the navigation subsystem. _What is the total syntax error score for those errors?_

In [2]:
from functools import reduce

def first_illegal(s):
    scores = {")": 3, "]": 57, "}": 1197, ">": 25137}
    for _ in s:
        s = s.replace('()', '').replace('[]','').replace('{}', '').replace('<>', '')
    if first_bad := next((c for c in s if c not in "([{<"), None):
        return scores[first_bad]
    return 0

def part1(filename):
    print(sum(first_illegal(line.strip()) for line in open(filename)))


part1("input/day10-test.txt")
part1("input/day10.txt")

26397

413733

<div class="alert alert-info">For a change this isn't the code I used which pushed and popped a stack and you can see
in the src directory. This code is a lot shorter: remove all matched pairs or brackets until there can't possibly be
any left then pull out the answer.
</div>

**Your puzzle answer was 413733.**

### Part Two

Now, discard the corrupted lines. The remaining lines are _incomplete_.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out _the sequence of closing characters_ that complete all open chunks in the line.

You can only use closing characters (`)`, `]`, `}`, or `>`), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In the example above, there are five incomplete lines:

*   `[({(<(())[]>[[{[]{<()<>>` - Complete by adding `}}]])})]`.
*   `[(()[<>])]({[<{<<[]>>(` - Complete by adding `)}>]})`.
*   `(((({<>}<{<{<>}{[]{[]{}` - Complete by adding `}}>}>))))`.
*   `{<[[]]>}<{[{[{[]{()[[[]` - Complete by adding `]]}}]}]}>`.
*   `<{([{{}}[<[[[<>{}]]]>[]]` - Complete by adding `])}>`.

Did you know that autocomplete tools _also_ have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of `0`. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

*   `)`: `1` point.
*   `]`: `2` points.
*   `}`: `3` points.
*   `>`: `4` points.

So, the last completion string above - `])}>` - would be scored as follows:

*   Start with a total score of `0`.
*   Multiply the total score by 5 to get `0`, then add the value of `]` (2) to get a new total score of `2`.
*   Multiply the total score by 5 to get `10`, then add the value of `)` (1) to get a new total score of `11`.
*   Multiply the total score by 5 to get `55`, then add the value of `}` (3) to get a new total score of `58`.
*   Multiply the total score by 5 to get `290`, then add the value of `>` (4) to get a new total score of `294`.

The five lines' completion strings have total scores as follows:

*   `}}]])})]` - `288957` total points.
*   `)}>]})` - `5566` total points.
*   `}}>}>))))` - `1480781` total points.
*   `]]}}]}]}>` - `995444` total points.
*   `])}>` - `294` total points.

Autocomplete tools are an odd bunch: the winner is found by _sorting_ all of the scores and then taking the _middle_ score. (There will always be an odd number of scores to consider.) In this example, the middle score is _`288957`_ because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. _What is the middle score?_

In [3]:
def incomplete(s):
    scores = {"(": 1, "[": 2, "{": 3, "<": 4}
    for _ in s:
        s = s.replace('()', '').replace('[]','').replace('{}', '').replace('<>', '')

    if next((c for c in s if c not in "([{<"), None):
            return 0

    return reduce(lambda a, b: a*5 + b, (scores[c] for c in s[::-1]), 0)


def part2(filename):
    scores = sorted(score for line in open(filename) if (score:=incomplete(line.strip())))
    print(scores[len(scores) // 2])


part2("input/day10-test.txt")
part2("input/day10.txt")

288957

3354640192

<div class="alert alert-info">And the same technique for part 2. Note the gratuitous use of the walrus operator.
</div>

**Your puzzle answer was 3354640192.**